In [1]:
!pip install optuna


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#Necessary imports

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from torch.nn.parameter import Parameter
from typing import Optional, Dict, Any, List
from torch import Tensor
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from sklearn import tree as sklearn_tree
import math
from typing import Any, Optional, cast
import sklearn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

openml_codes = {'gesture':4538, 'house': 574}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Data preparation toolkit


In [2]:
def get_XY(name):
  data_id = openml_codes[name]
  bunch = cast(
    sklearn.utils.Bunch,
    sklearn.datasets.fetch_openml(data_id=data_id, as_frame=True),
)
  X = bunch.data.values.astype(np.float32)
  if name == 'gesture':
    Y = bunch.target.cat.codes.values.astype(np.int32)
  elif name=='house':
    Y = bunch.target.values.astype(np.float32)
  return X, Y

In [ ]:
def _check_bins(bins: List[Tensor]) -> None:
    if not bins:
        raise ValueError('The list of bins must not be empty')
    for i, feature_bins in enumerate(bins):
        if not isinstance(feature_bins, Tensor):
            raise ValueError(
                'bins must be a list of PyTorch tensors. '
                f'However, for {i=}: {type(bins[i])=}'
            )
        if feature_bins.ndim != 1:
            raise ValueError(
                'Each item of the bin list must have exactly one dimension.'
                f' However, for {i=}: {bins[i].ndim=}'
            )
        if len(feature_bins) < 2:
            raise ValueError(
                'All features must have at least two bin edges.'
                f' However, for {i=}: {len(bins[i])=}'
            )
        if not feature_bins.isfinite().all():
            raise ValueError(
                'Bin edges must not contain nan/inf/-inf.'
                f' However, this is not true for the {i}-th feature'
            )
        if (feature_bins[:-1] >= feature_bins[1:]).any():
            raise ValueError(
                'Bin edges must be sorted.'
                f' However, the for the {i}-th feature, the bin edges are not sorted'
            )
        if len(feature_bins) == 2:
            warnings.warn(
                f'The {i}-th feature has just two bin edges, which means only one bin.'
                ' Strictly speaking, using a single bin for the'
                ' piecewise-linear encoding should not break anything,'
                ' but it is the same as using sklearn.preprocessing.MinMaxScaler'
            )
def compute_bins(
    X: torch.Tensor,
    n_bins: int = 48,
    *,
    tree_kwargs: Optional[Dict[str, Any]] = None,
    y: Optional[Tensor] = None,
    regression: Optional[bool] = None,
    verbose: bool = False,
) -> List[Tensor]:
    """Compute bin edges for `PiecewiseLinearEmbeddings`.

    **Usage**

    Computing the quantile-based bins (Section 3.2.1 in the paper):

    >>> X_train = torch.randn(10000, 2)
    >>> bins = compute_bins(X_train)

    Computing the tree-based bins (Section 3.2.2 in the paper):

    >>> X_train = torch.randn(10000, 2)
    >>> y_train = torch.randn(len(X_train))
    >>> bins = compute_bins(
    ...     X_train,
    ...     y=y_train,
    ...     regression=True,
    ...     tree_kwargs={'min_samples_leaf': 64, 'min_impurity_decrease': 1e-4},
    ... )

    Args:
        X: the training features.
        n_bins: the number of bins.
        tree_kwargs: keyword arguments for `sklearn.tree.DecisionTreeRegressor`
            (if ``regression`` is `True`) or `sklearn.tree.DecisionTreeClassifier`
            (if ``regression`` is `False`).
            NOTE: requires ``scikit-learn>=1.0,>2`` to be installed.
        y: the training labels (must be provided if ``tree`` is not None).
        regression: whether the labels are regression labels
            (must be provided if ``tree`` is not None).
        verbose: if True and ``tree_kwargs`` is not None, than ``tqdm``
            (must be installed) will report the progress while fitting trees.
    Returns:
        A list of bin edges for all features. For one feature:

        - the maximum possible number of bin edges is ``n_bins + 1``.
        - the minumum possible number of bin edges is ``1``.
    """
    if not isinstance(X, Tensor):
        raise ValueError(f'X must be a PyTorch tensor, however: {type(X)=}')
    if X.ndim != 2:
        raise ValueError(f'X must have exactly two dimensions, however: {X.ndim=}')
    if X.shape[0] < 2:
        raise ValueError(f'X must have at least two rows, however: {X.shape[0]=}')
    if X.shape[1] < 1:
        raise ValueError(f'X must have at least one column, however: {X.shape[1]=}')
    if not X.isfinite().all():
        raise ValueError('X must not contain nan/inf/-inf.')
    if (X == X[0]).all(dim=0).any():
        raise ValueError(
            'All columns of X must have at least two distinct values.'
            ' However, X contains columns with just one distinct value.'
        )
    if n_bins <= 1 or n_bins >= len(X):
        raise ValueError(
            'n_bins must be more than 1, but less than len(X), however:'
            f' {n_bins=}, {len(X)=}'
        )

    if tree_kwargs is None:
        if y is not None or regression is not None or verbose:
            raise ValueError(
                'If tree_kwargs is None, then y must be None, regression must be None'
                ' and verbose must be False'
            )

        bins = [
            q.unique()
            for q in torch.quantile(
                X, torch.linspace(0.0, 1.0, n_bins + 1).to(X), dim=0
            ).T
        ]
        _check_bins(bins)
        return bins
    else:
        if sklearn_tree is None:
            raise RuntimeError(
                'The scikit-learn package is missing.'
                ' See README.md for installation instructions'
            )
        if y is None or regression is None:
            raise ValueError(
                'If tree_kwargs is not None, then y and regression must not be None'
            )
        if y.ndim != 1:
            raise ValueError(f'y must have exactly one dimension, however: {y.ndim=}')
        if len(y) != len(X):
            raise ValueError(
                f'len(y) must be equal to len(X), however: {len(y)=}, {len(X)=}'
            )
        if y is None or regression is None:
            raise ValueError(
                'If tree_kwargs is not None, then y and regression must not be None'
            )
        if 'max_leaf_nodes' in tree_kwargs:
            raise ValueError(
                'tree_kwargs must not contain the key "max_leaf_nodes"'
                ' (it will be set to n_bins automatically).'
            )

        if verbose:
            if tqdm is None:
                raise ImportError('If verbose is True, tqdm must be installed')
            tqdm_ = tqdm
        else:
            tqdm_ = lambda x: x  # noqa: E731

        if X.device.type != 'cpu' or y.device.type != 'cpu':
            warnings.warn(
                'Computing tree-based bins involves the conversion of the input PyTorch'
                ' tensors to NumPy arrays. The provided PyTorch tensors are not'
                ' located on CPU, so the conversion has some overhead.',
                UserWarning,
            )
        X_numpy = X.cpu().numpy()
        y_numpy = y.cpu().numpy()
        bins = []
        for column in tqdm_(X_numpy.T):
            feature_bin_edges = [float(column.min()), float(column.max())]
            tree = (
                (
                    sklearn_tree.DecisionTreeRegressor
                    if regression
                    else sklearn_tree.DecisionTreeClassifier
                )(max_leaf_nodes=n_bins, **tree_kwargs)
                .fit(column.reshape(-1, 1), y_numpy)
                .tree_
            )
            for node_id in range(tree.node_count):
                # The following condition is True only for split nodes. Source:
                # https://scikit-learn.org/1.0/auto_examples/tree/plot_unveil_tree_structure.html#tree-structure
                if tree.children_left[node_id] != tree.children_right[node_id]:
                    feature_bin_edges.append(float(tree.threshold[node_id]))
            bins.append(torch.as_tensor(feature_bin_edges).unique())
        _check_bins(bins)
        return [x.to(device=X.device, dtype=X.dtype) for x in bins]

def emb_ple(x, bin):
    x_emb = np.zeros(len(bin)-1)
    for i in range(len(x_emb)):
        if (x < bin[i]) and (i > 0):
            x_emb[i] = 0
        elif  (x >= bin[i+1]) and (i < len(x_emb)):
            x_emb[i] = 1
        else:
             x_emb[i] = (x - bin[i])/(bin[i+1] - bin[i])
    return x_emb

def ple(bins, x):
    bin = bins[0].numpy() # for torch tensors
    #bin = bins[0]
    f1 = emb_ple(x[0, 0], bin)
    for k in range(len(x)-1):
        f2 = emb_ple(x[k+1, 0], bin)
        f1 = np.vstack([f1, f2])
    x_ple = f1
    for i in range(len(x[0])-1):
        bin = bins[i+1].numpy()
        #bin = bins[i+1]
        f1 = emb_ple(x[0, i+1], bin)
        for k in range(len(x)-1):
            f2 = emb_ple(x[k+1, i+1], bin)
            f1 = np.vstack([f1, f2])
        x_ple = np.concatenate([x_ple, f1], axis = 1)
    return x_ple

def periodic_embedding(X, n_features):
  x = torch.from_numpy(X)
  k = 48
  sigma = 0.01
  weight = Parameter(torch.empty(n_features, k))
  x = 2 * math.pi * weight * x[..., None]
  x = torch.cat([torch.cos(x), torch.sin(x)], -1)
  x = x.detach().numpy()
  x_f = x[:, 0, :]
  for i in range(x.shape[1]-1):
    x_f = np.concatenate([x_f, x[:, i+1, :]], axis = 1)

  return x_f

In [3]:
def PLE_q(X_train, X_val, X_test):
  bins = compute_bins(torch.from_numpy(X_train))#ple-q
  x_train_emb = ple(bins, X_train)
  x_val_emb = ple(bins, X_val)
  x_test_emb = ple(bins, X_test)
  return x_train_emb, x_val_emb, x_test_emb
def PLE_t(X_train, X_val, X_test):
  bins = compute_bins(
    torch.from_numpy(X_train),
    tree_kwargs={'min_samples_leaf': 64, 'min_impurity_decrease': 1e-4},
    y=torch.from_numpy(y_train),
    regression=True,
  )
  x_train_emb = ple(bins, X_train)
  x_val_emb = ple(bins, X_val)
  x_test_emb = ple(bins, X_test)
  return x_train_emb, x_val_emb, x_test_emb

#MLP structure

In [7]:
class MLP(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate, learning_rate, weight_decay):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()

        for _ in range(num_layers):
            self.layers.append(nn.Sequential(
                nn.Linear(input_size, num_neurons),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ))
            input_size = num_neurons

        self.fc = nn.Linear(num_neurons, 1)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.fc(x)
        return x

def objective(trial):
    # Define hyperparameters to search
    learning_rate = trial.suggest_float('learning_rate', 5e-5, 0.003, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-8, 0.0001, log=True)
    num_layers = trial.suggest_int('num_layers', 3, 6)
    num_neurons = trial.suggest_int('num_neurons', 16, 1024)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)

    # Create the neural network
    model = MLP(input_size=X_train.shape[1], num_layers=num_layers, num_neurons=num_neurons, dropout_rate=dropout_rate, learning_rate=learning_rate, weight_decay=weight_decay)
    model = model.to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)


    # Train the neural network
    for epoch in range(200):
        optimizer.zero_grad()
        output = model(torch.Tensor(X_train))
        loss = criterion(output.squeeze(), torch.Tensor(y_train))
        loss.backward()
        optimizer.step()

    # Evaluate the model on the validation set
    with torch.no_grad():
        output = model(torch.Tensor(X_val))
        val_loss = criterion(output.squeeze(), torch.Tensor(y_val))

    return np.sqrt(val_loss.item())
# Train the best model on the entire training set
def calculate_metrics(best_params):
  best_model = MLP(input_size=X_train.shape[1], **best_params)
  best_model = best_model.to(device)

  criterion = nn.MSELoss()
  optimizer = optim.AdamW(best_model.parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'])

  best_estimation={'loss': 999999, 'epoch_number': 999}
  # Train the best model without early stopping
  for epoch in tqdm(range(400), desc="Epochs"):
      optimizer.zero_grad()
      output = best_model(X_train)
      loss = criterion(output.squeeze(), y_train)
      loss.backward()
      optimizer.step()

      with torch.no_grad():
          output = best_model(X_test)
          val_loss = np.sqrt((criterion(output.squeeze(), y_test)).item())
          print(f"Epoch {epoch + 1}, Test Loss: {val_loss:.4f}")
          if val_loss<best_estimation['loss']:
            best_estimation['loss'] = val_loss
            best_estimation['epoch_number'] = epoch
  print(f'Minimal loss {best_estimation["loss"]:.3f} was observed on test data on {best_estimation["epoch_number"]} epoch')

def calculate_metrics(best_params):
  best_model = MLP(input_size=X_train.shape[1], **best_params)
  best_model = best_model.to(device)

  criterion = nn.MSELoss()
  optimizer = optim.AdamW(best_model.parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'])

  best_estimation={'loss': 99999, 'epoch_number': 999}
  # Train the best model without early stopping
  for epoch in tqdm(range(400), desc="Epochs"):
      optimizer.zero_grad()
      output = best_model(X_train)
      loss = criterion(output.squeeze(), y_train)
      loss.backward()
      optimizer.step()

      with torch.no_grad():
          output = best_model(X_test)
          val_loss = np.sqrt((criterion(output.squeeze(), y_test)).item())
          print(f"Epoch {epoch + 1}, Test Loss: {val_loss:.4f}")
          if val_loss<best_estimation['loss']:
            best_estimation['loss'] = val_loss
            best_estimation['epoch_number'] = epoch
  print(f'Minimal loss {best_estimation["loss"]:.3f} was observed on test data on {best_estimation["epoch_number"]} epoch')

#California dataset

In [ ]:
california = fetch_california_housing()
X = california.data
y = california.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

##MLP without embeddings

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_train = torch.from_numpy(X_train).to(device).float()
y_train = torch.from_numpy(y_train).to(device).float()
X_val = torch.from_numpy(X_val).to(device).float()
y_val = torch.from_numpy(y_val).to(device).float()
X_test = torch.from_numpy(X_test).to(device).float()
y_test = torch.from_numpy(y_test).to(device).float()
# Create an Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

best_params = study.best_params
print("Best hyperparameters:", best_params)
print("------------------------------------------------")
calculate_metrics(best_params)

[I 2024-03-20 21:11:59,877] A new study created in memory with name: no-name-56a2e517-7dbf-4998-b938-10fb74c9c454
[I 2024-03-20 21:12:05,171] Trial 0 finished with value: 0.5993782616965099 and parameters: {'learning_rate': 0.0007285238275878853, 'weight_decay': 6.764027821257639e-05, 'num_layers': 6, 'num_neurons': 240, 'dropout_rate': 0.06417480252450158}. Best is trial 0 with value: 0.5993782616965099.
[I 2024-03-20 21:12:11,207] Trial 1 finished with value: 0.6164492283911807 and parameters: {'learning_rate': 0.00020175767405705372, 'weight_decay': 9.033302062773524e-05, 'num_layers': 6, 'num_neurons': 504, 'dropout_rate': 0.07893567504097498}. Best is trial 0 with value: 0.5993782616965099.
[I 2024-03-20 21:12:18,213] Trial 2 finished with value: 0.6588616628308364 and parameters: {'learning_rate': 0.0020315373010161993, 'weight_decay': 1.0522761921991872e-08, 'num_layers': 5, 'num_neurons': 620, 'dropout_rate': 0.24548736641856078}. Best is trial 0 with value: 0.5993782616965099.

Best hyperparameters: {'learning_rate': 0.0015708798739493383, 'weight_decay': 1.1197964622487542e-05, 'num_layers': 6, 'num_neurons': 267, 'dropout_rate': 0.0006393841689775232}
------------------------------------------------


Epochs:   2%|▏         | 7/400 [00:00<00:06, 61.95it/s]

Epoch 1, Test Loss: 2.2600
Epoch 2, Test Loss: 2.1201
Epoch 3, Test Loss: 1.8462
Epoch 4, Test Loss: 1.6316
Epoch 5, Test Loss: 2.7279
Epoch 6, Test Loss: 1.7766
Epoch 7, Test Loss: 1.4523
Epoch 8, Test Loss: 1.5014
Epoch 9, Test Loss: 1.5550
Epoch 10, Test Loss: 1.5460
Epoch 11, Test Loss: 1.4806
Epoch 12, Test Loss: 1.3534
Epoch 13, Test Loss: 1.1890


Epochs:   5%|▌         | 21/400 [00:00<00:06, 60.63it/s]

...

Epoch 375, Test Loss: 0.5560
Epoch 376, Test Loss: 0.5694
Epoch 377, Test Loss: 0.5380
Epoch 378, Test Loss: 0.5512
Epoch 379, Test Loss: 0.5333
Epoch 380, Test Loss: 0.5708
Epoch 381, Test Loss: 0.5354
Epoch 382, Test Loss: 0.5420
Epoch 383, Test Loss: 0.5539
Epoch 384, Test Loss: 0.5349
Epoch 385, Test Loss: 0.5410
Epoch 386, Test Loss: 0.5333
Epoch 387, Test Loss: 0.5492


Epochs: 100%|█████████▉| 399/400 [00:06<00:00, 59.25it/s]

Epoch 388, Test Loss: 0.5380
Epoch 389, Test Loss: 0.5365
Epoch 390, Test Loss: 0.5543
Epoch 391, Test Loss: 0.5370
Epoch 392, Test Loss: 0.5327
Epoch 393, Test Loss: 0.5540
Epoch 394, Test Loss: 0.5349
Epoch 395, Test Loss: 0.5324
Epoch 396, Test Loss: 0.5523
Epoch 397, Test Loss: 0.5347
Epoch 398, Test Loss: 0.5323
Epoch 399, Test Loss: 0.5489


Epochs: 100%|██████████| 400/400 [00:06<00:00, 59.47it/s]

Epoch 400, Test Loss: 0.5369
Minimal loss 0.525 was observed on test data on 272 epoch


##MLP-q

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

x_train_emb, x_val_emb, x_test_emb = PLE_q(X_train, X_val, X_test)

X_train = torch.from_numpy(x_train_emb).to(device).float()
y_train = torch.from_numpy(y_train).to(device).float()
X_val = torch.from_numpy(x_val_emb).to(device).float()
y_val = torch.from_numpy(y_val).to(device).float()
X_test = torch.from_numpy(x_test_emb).to(device).float()
y_test = torch.from_numpy(y_test).to(device).float()
# Create an Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

best_params = study.best_params
print("Best hyperparameters:", best_params)
print("------------------------------------------------")
calculate_metrics(best_params)

[I 2024-03-20 21:35:42,166] A new study created in memory with name: no-name-bd7246ed-e8bc-4bd1-a922-551330182914
[I 2024-03-20 21:35:51,918] Trial 0 finished with value: 0.5605048452962478 and parameters: {'learning_rate': 0.001662644531805776, 'weight_decay': 2.2870873195227012e-08, 'num_layers': 5, 'num_neurons': 641, 'dropout_rate': 0.09966685644870882}. Best is trial 0 with value: 0.5605048452962478.
[I 2024-03-20 21:35:56,089] Trial 1 finished with value: 0.5936279924421299 and parameters: {'learning_rate': 0.00016096807669483388, 'weight_decay': 1.8969099698953087e-05, 'num_layers': 4, 'num_neurons': 472, 'dropout_rate': 0.3058637828766197}. Best is trial 0 with value: 0.5605048452962478.
[I 2024-03-20 21:35:57,047] Trial 2 finished with value: 0.9804738261178827 and parameters: {'learning_rate': 9.810112833483918e-05, 'weight_decay': 2.776129966117187e-06, 'num_layers': 6, 'num_neurons': 128, 'dropout_rate': 0.49443752349351083}. Best is trial 0 with value: 0.5605048452962478.


Best hyperparameters: {'learning_rate': 0.0005443306022735116, 'weight_decay': 5.348540820279637e-07, 'num_layers': 3, 'num_neurons': 727, 'dropout_rate': 0.0007160031610542105}
------------------------------------------------


Epochs:   1%|          | 4/400 [00:00<00:13, 29.62it/s]

Epoch 1, Test Loss: 1.9943
Epoch 2, Test Loss: 1.5597
Epoch 3, Test Loss: 1.1535
Epoch 4, Test Loss: 1.6206
Epoch 5, Test Loss: 1.3577
Epoch 6, Test Loss: 1.0968


Epochs:   2%|▎         | 10/400 [00:00<00:13, 28.17it/s]

...

Epoch 384, Test Loss: 0.5138
Epoch 385, Test Loss: 0.5024
Epoch 386, Test Loss: 0.5023
Epoch 387, Test Loss: 0.5071
Epoch 388, Test Loss: 0.5073
Epoch 389, Test Loss: 0.5204


Epochs:  98%|█████████▊| 394/400 [00:14<00:00, 27.41it/s]

Epoch 390, Test Loss: 0.5167
Epoch 391, Test Loss: 0.5303
Epoch 392, Test Loss: 0.5233
Epoch 393, Test Loss: 0.5264
Epoch 394, Test Loss: 0.5089
Epoch 395, Test Loss: 0.5073


Epochs: 100%|██████████| 400/400 [00:14<00:00, 27.40it/s]

Epoch 396, Test Loss: 0.5029
Epoch 397, Test Loss: 0.5039
Epoch 398, Test Loss: 0.5161
Epoch 399, Test Loss: 0.5123
Epoch 400, Test Loss: 0.5205
Minimal loss 0.485 was observed on test data on 200 epoch


##MLP-t

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

x_train_emb, x_val_emb, x_test_emb = PLE_t(X_train, X_val, X_test)

X_train = torch.from_numpy(x_train_emb).to(device).float()
y_train = torch.from_numpy(y_train).to(device).float()
X_val = torch.from_numpy(x_val_emb).to(device).float()
y_val = torch.from_numpy(y_val).to(device).float()
X_test = torch.from_numpy(x_test_emb).to(device).float()
y_test = torch.from_numpy(y_test).to(device).float()
# Create an Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

best_params = study.best_params
print("Best hyperparameters:", best_params)
print("------------------------------------------------")
calculate_metrics(best_params)

[I 2024-03-20 22:24:40,943] A new study created in memory with name: no-name-da4800a8-c6ea-45c7-b83a-8576217d5755
[I 2024-03-20 22:24:51,138] Trial 0 finished with value: 0.6798000680948119 and parameters: {'learning_rate': 0.0023787299275336957, 'weight_decay': 1.2652261647033981e-08, 'num_layers': 6, 'num_neurons': 632, 'dropout_rate': 0.4357833005881382}. Best is trial 0 with value: 0.6798000680948119.
[I 2024-03-20 22:24:53,312] Trial 1 finished with value: 0.592395693534869 and parameters: {'learning_rate': 0.0022248197558172207, 'weight_decay': 7.85553661152296e-06, 'num_layers': 3, 'num_neurons': 353, 'dropout_rate': 0.29798513703294904}. Best is trial 1 with value: 0.592395693534869.
[I 2024-03-20 22:25:04,239] Trial 2 finished with value: 0.5166096100784301 and parameters: {'learning_rate': 0.0013553361553034114, 'weight_decay': 8.718558991545767e-06, 'num_layers': 3, 'num_neurons': 1003, 'dropout_rate': 0.035558229703339383}. Best is trial 2 with value: 0.5166096100784301.
[I

Best hyperparameters: {'learning_rate': 0.0001867251403339556, 'weight_decay': 5.507974011922566e-06, 'num_layers': 3, 'num_neurons': 932, 'dropout_rate': 0.00029123778133175425}
------------------------------------------------


Epochs:   0%|          | 2/400 [00:00<00:21, 18.82it/s]

Epoch 1, Test Loss: 2.1775
Epoch 2, Test Loss: 2.0106
Epoch 3, Test Loss: 1.8256


Epochs:   1%|          | 4/400 [00:00<00:22, 17.40it/s]

...

Epoch 396, Test Loss: 0.5021


Epochs: 100%|█████████▉| 398/400 [00:24<00:00, 16.45it/s]

Epoch 397, Test Loss: 0.5018
Epoch 398, Test Loss: 0.5028
Epoch 399, Test Loss: 0.5024


Epochs: 100%|██████████| 400/400 [00:24<00:00, 16.57it/s]

Epoch 400, Test Loss: 0.5049
Minimal loss 0.490 was observed on test data on 246 epoch


##MLP-p

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

x_train_emb = periodic_embedding(X_train, X_train.shape[1])
x_val_emb = periodic_embedding(X_val, X_val.shape[1])
x_test_emb = periodic_embedding(X_test, X_test.shape[1])


X_train = torch.from_numpy(x_train_emb).to(device).float()
y_train = torch.from_numpy(y_train).to(device).float()
X_val = torch.from_numpy(x_val_emb).to(device).float()
y_val = torch.from_numpy(y_val).to(device).float()
X_test = torch.from_numpy(x_test_emb).to(device).float()
y_test = torch.from_numpy(y_test).to(device).float()
# Create an Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

best_params = study.best_params
print("Best hyperparameters:", best_params)
print("------------------------------------------------")
calculate_metrics(best_params)

[I 2024-03-20 23:21:15,357] A new study created in memory with name: no-name-fa544ec1-27f1-42ff-b25c-96476a515bae
[I 2024-03-20 23:21:25,439] Trial 0 finished with value: 1.1414130114721286 and parameters: {'learning_rate': 0.00017164677166178386, 'weight_decay': 1.2143225401423021e-05, 'num_layers': 6, 'num_neurons': 616, 'dropout_rate': 0.0694858029669086}. Best is trial 0 with value: 1.1414130114721286.
[I 2024-03-20 23:21:25,995] Trial 1 finished with value: 1.1955963495852866 and parameters: {'learning_rate': 5.2644633420837286e-05, 'weight_decay': 1.685728410365796e-08, 'num_layers': 6, 'num_neurons': 47, 'dropout_rate': 0.034939446805328345}. Best is trial 0 with value: 1.1414130114721286.
[I 2024-03-20 23:21:30,515] Trial 2 finished with value: 1.163337328028629 and parameters: {'learning_rate': 0.00014999197741479527, 'weight_decay': 5.914754128380322e-06, 'num_layers': 4, 'num_neurons': 465, 'dropout_rate': 0.39286282878025697}. Best is trial 0 with value: 1.1414130114721286.

Best hyperparameters: {'learning_rate': 0.0007847380475814592, 'weight_decay': 2.8100724285068097e-05, 'num_layers': 6, 'num_neurons': 1007, 'dropout_rate': 0.015474165784184257}
------------------------------------------------


Epochs:   0%|          | 1/400 [00:00<00:53,  7.43it/s]

Epoch 1, Test Loss: 2.1576


Epochs:   0%|          | 2/400 [00:00<00:56,  7.05it/s]

...

Epoch 398, Test Loss: 1.1123


Epochs: 100%|█████████▉| 399/400 [00:56<00:00,  7.04it/s]

Epoch 399, Test Loss: 1.0659


Epochs: 100%|██████████| 400/400 [00:56<00:00,  7.05it/s]

Epoch 400, Test Loss: 1.1062
Minimal loss 1.044 was observed on test data on 269 epoch


#House16 dataset

In [9]:
X, y = get_XY("house")
scaler = StandardScaler()
X = scaler.fit_transform(X)

D:\Python311\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


##MLP without embeddings

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_train = torch.from_numpy(X_train).to(device).float()
y_train = torch.from_numpy(y_train).to(device).float()
X_val = torch.from_numpy(X_val).to(device).float()
y_val = torch.from_numpy(y_val).to(device).float()
X_test = torch.from_numpy(X_test).to(device).float()
y_test = torch.from_numpy(y_test).to(device).float()
# Create an Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

best_params = study.best_params
print("Best hyperparameters:", best_params)
print("------------------------------------------------")
calculate_metrics(best_params)

[I 2024-03-21 02:58:35,610] A new study created in memory with name: no-name-db64a8bd-74af-4dd6-bf1c-473c3acb406d
[I 2024-03-21 02:58:45,018] Trial 0 finished with value: 31818.425856726477 and parameters: {'learning_rate': 0.002535677709764444, 'weight_decay': 1.7719352310673227e-06, 'num_layers': 6, 'num_neurons': 531, 'dropout_rate': 0.06314069321522836}. Best is trial 0 with value: 31818.425856726477.
[I 2024-03-21 02:58:45,793] Trial 1 finished with value: 72677.45642219463 and parameters: {'learning_rate': 7.513260030434804e-05, 'weight_decay': 1.0684298675705799e-05, 'num_layers': 3, 'num_neurons': 180, 'dropout_rate': 0.24525477401350915}. Best is trial 0 with value: 31818.425856726477.
[I 2024-03-21 02:58:46,430] Trial 2 finished with value: 52816.814292420175 and parameters: {'learning_rate': 0.0007791745896109273, 'weight_decay': 7.041472120293669e-07, 'num_layers': 3, 'num_neurons': 152, 'dropout_rate': 0.07654522684442744}. Best is trial 0 with value: 31818.425856726477.
[

Best hyperparameters: {'learning_rate': 0.002071283009818489, 'weight_decay': 1.4082965915795697e-08, 'num_layers': 6, 'num_neurons': 801, 'dropout_rate': 0.00031217916420113803}
------------------------------------------------


Epochs:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch 1, Test Loss: 73516.6798


Epochs:   0%|          | 2/400 [00:00<00:38, 10.38it/s]

...

Epoch 395, Test Loss: 35394.5496
Epoch 396, Test Loss: 34206.3087


Epochs: 100%|█████████▉| 398/400 [00:40<00:00,  9.76it/s]

Epoch 397, Test Loss: 36216.5257
Epoch 398, Test Loss: 35225.8814


Epochs: 100%|██████████| 400/400 [00:40<00:00,  9.76it/s]

Epoch 399, Test Loss: 34786.7156
Epoch 400, Test Loss: 36312.2582
Minimal loss 32918.292 was observed on test data on 214 epoch


##MLP-q

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

x_train_emb, x_val_emb, x_test_emb = PLE_q(X_train, X_val, X_test)

X_train = torch.from_numpy(x_train_emb).to(device).float()
y_train = torch.from_numpy(y_train).to(device).float()
X_val = torch.from_numpy(x_val_emb).to(device).float()
y_val = torch.from_numpy(y_val).to(device).float()
X_test = torch.from_numpy(x_test_emb).to(device).float()
y_test = torch.from_numpy(y_test).to(device).float()
# Create an Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

best_params = study.best_params
print("Best hyperparameters:", best_params)
print("------------------------------------------------")
calculate_metrics(best_params)

[I 2024-03-21 13:59:32,444] A new study created in memory with name: no-name-8d939540-c824-4626-8601-c1435aa34dd0
[I 2024-03-21 13:59:36,367] Trial 0 finished with value: 35168.78070107066 and parameters: {'learning_rate': 0.0029712693074799017, 'weight_decay': 2.3019617018303453e-08, 'num_layers': 4, 'num_neurons': 139, 'dropout_rate': 0.13849302808704994}. Best is trial 0 with value: 35168.78070107066.
[I 2024-03-21 13:59:51,568] Trial 1 finished with value: 34249.64163316165 and parameters: {'learning_rate': 0.0005097140977606904, 'weight_decay': 4.732776596533437e-08, 'num_layers': 4, 'num_neurons': 936, 'dropout_rate': 0.09390145916552078}. Best is trial 1 with value: 34249.64163316165.
[I 2024-03-21 13:59:57,839] Trial 2 finished with value: 52360.241405096676 and parameters: {'learning_rate': 6.368065281351536e-05, 'weight_decay': 1.1255356667185071e-05, 'num_layers': 4, 'num_neurons': 518, 'dropout_rate': 0.2112569352866931}. Best is trial 1 with value: 34249.64163316165.
[I 20

Best hyperparameters: {'learning_rate': 0.0006067413434629957, 'weight_decay': 9.329171979934277e-07, 'num_layers': 6, 'num_neurons': 1016, 'dropout_rate': 0.016782168590207383}
------------------------------------------------


Epochs:   0%|          | 1/400 [00:00<00:55,  7.13it/s]

Epoch 1, Test Loss: 73517.9856


Epochs:   0%|          | 2/400 [00:00<00:58,  6.83it/s]

...

Epoch 398, Test Loss: 33396.7491


Epochs: 100%|█████████▉| 399/400 [01:01<00:00,  6.59it/s]

Epoch 399, Test Loss: 33467.2004


Epochs: 100%|██████████| 400/400 [01:01<00:00,  6.51it/s]

Epoch 400, Test Loss: 33373.6624
Minimal loss 31979.200 was observed on test data on 260 epoch


##MLP-t

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

x_train_emb, x_val_emb, x_test_emb = PLE_t(X_train, X_val, X_test)

X_train = torch.from_numpy(x_train_emb).to(device).float()
y_train = torch.from_numpy(y_train).to(device).float()
X_val = torch.from_numpy(x_val_emb).to(device).float()
y_val = torch.from_numpy(y_val).to(device).float()
X_test = torch.from_numpy(x_test_emb).to(device).float()
y_test = torch.from_numpy(y_test).to(device).float()
# Create an Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

best_params = study.best_params
print("Best hyperparameters:", best_params)
print("------------------------------------------------")
calculate_metrics(best_params)

[I 2024-03-21 16:14:32,506] A new study created in memory with name: no-name-0a011e7b-b70f-46fd-8352-9dcc794ecc4e
[I 2024-03-21 16:14:45,344] Trial 0 finished with value: 33158.62675081705 and parameters: {'learning_rate': 0.002299858142536019, 'weight_decay': 2.610450286226054e-07, 'num_layers': 6, 'num_neurons': 541, 'dropout_rate': 0.39347375032437537}. Best is trial 0 with value: 33158.62675081705.
[I 2024-03-21 16:14:47,101] Trial 1 finished with value: 71935.57295246907 and parameters: {'learning_rate': 5.102369391724746e-05, 'weight_decay': 2.6028101725134105e-08, 'num_layers': 4, 'num_neurons': 210, 'dropout_rate': 0.12318316545359193}. Best is trial 0 with value: 33158.62675081705.
[I 2024-03-21 16:14:50,889] Trial 2 finished with value: 57174.709933676095 and parameters: {'learning_rate': 9.011175039727064e-05, 'weight_decay': 6.933485809290222e-07, 'num_layers': 3, 'num_neurons': 467, 'dropout_rate': 0.37592608756850737}. Best is trial 0 with value: 33158.62675081705.
[I 202

Best hyperparameters: {'learning_rate': 0.001591021982962098, 'weight_decay': 2.0761141651952226e-05, 'num_layers': 5, 'num_neurons': 789, 'dropout_rate': 0.1456151209075444}
------------------------------------------------


Epochs:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch 1, Test Loss: 73516.9723


Epochs:   0%|          | 2/400 [00:00<00:33, 11.78it/s]

...

Epoch 394, Test Loss: 36136.1355
Epoch 395, Test Loss: 35188.6363
Epoch 396, Test Loss: 35053.0350


Epochs: 100%|█████████▉| 398/400 [00:36<00:00, 10.81it/s]

Epoch 397, Test Loss: 34770.0414
Epoch 398, Test Loss: 34934.9072
Epoch 399, Test Loss: 35296.0091


Epochs: 100%|██████████| 400/400 [00:36<00:00, 10.89it/s]

Epoch 400, Test Loss: 34914.8468
Minimal loss 32783.313 was observed on test data on 187 epoch


##MLP-p

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

x_train_emb = periodic_embedding(X_train, X_train.shape[1])
x_val_emb = periodic_embedding(X_val, X_val.shape[1])
x_test_emb = periodic_embedding(X_test, X_test.shape[1])


X_train = torch.from_numpy(x_train_emb).to(device).float()
y_train = torch.from_numpy(y_train).to(device).float()
X_val = torch.from_numpy(x_val_emb).to(device).float()
y_val = torch.from_numpy(y_val).to(device).float()
X_test = torch.from_numpy(x_test_emb).to(device).float()
y_test = torch.from_numpy(y_test).to(device).float()
# Create an Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

best_params = study.best_params
print("Best hyperparameters:", best_params)
print("------------------------------------------------")
calculate_metrics(best_params)

[I 2024-03-21 20:27:08,348] A new study created in memory with name: no-name-d929f94c-1cc9-4d1c-a480-93fc92bf895f
[I 2024-03-21 20:27:22,548] Trial 0 finished with value: 52456.88515342862 and parameters: {'learning_rate': 0.0008608689035839534, 'weight_decay': 2.407427769462605e-05, 'num_layers': 3, 'num_neurons': 487, 'dropout_rate': 0.27719262765354663}. Best is trial 0 with value: 52456.88515342862.
[I 2024-03-21 20:27:44,287] Trial 1 finished with value: 52425.28367114478 and parameters: {'learning_rate': 0.00010905990557154158, 'weight_decay': 1.6559286669466016e-05, 'num_layers': 5, 'num_neurons': 507, 'dropout_rate': 0.46150885380209433}. Best is trial 1 with value: 52425.28367114478.
[I 2024-03-21 20:28:32,507] Trial 2 finished with value: 52316.61885099227 and parameters: {'learning_rate': 0.0001201791723636075, 'weight_decay': 1.473813819851223e-08, 'num_layers': 4, 'num_neurons': 944, 'dropout_rate': 0.0850339990531655}. Best is trial 2 with value: 52316.61885099227.
[I 202

[I 2024-03-21 20:42:34,920] Trial 28 finished with value: 52300.07204583948 and parameters: {'learning_rate': 0.0003645376148706045, 'weight_decay': 4.4269499194528385e-06, 'num_layers': 4, 'num_neurons': 397, 'dropout_rate': 0.02966884179433547}. Best is trial 12 with value: 52287.8873927796.
[I 2024-03-21 20:42:53,977] Trial 29 finished with value: 54906.32604718695 and parameters: {'learning_rate': 0.0011336860940995858, 'weight_decay': 1.106689777640429e-05, 'num_layers': 5, 'num_neurons': 394, 'dropout_rate': 0.2682102848922018}. Best is trial 12 with value: 52287.8873927796.
[I 2024-03-21 20:42:59,664] Trial 30 finished with value: 52438.94979878983 and parameters: {'learning_rate': 0.00034095337729935724, 'weight_decay': 1.4067587838849213e-06, 'num_layers': 3, 'num_neurons': 223, 'dropout_rate': 0.15326573443661584}. Best is trial 12 with value: 52287.8873927796.
[I 2024-03-21 20:43:16,631] Trial 31 finished with value: 52336.777738030454 and parameters: {'learning_rate': 0.000

[I 2024-03-21 20:48:25,660] Trial 56 finished with value: 52321.242798695064 and parameters: {'learning_rate': 0.001971079694896253, 'weight_decay': 4.10890171186979e-07, 'num_layers': 3, 'num_neurons': 177, 'dropout_rate': 0.000288940518812095}. Best is trial 40 with value: 52275.761113540946.
[I 2024-03-21 20:48:28,807] Trial 57 finished with value: 52418.00240375438 and parameters: {'learning_rate': 0.0011569923652277364, 'weight_decay': 1.5206039628499883e-06, 'num_layers': 3, 'num_neurons': 125, 'dropout_rate': 0.10648648011192877}. Best is trial 40 with value: 52275.761113540946.
[I 2024-03-21 20:48:34,810] Trial 58 finished with value: 52701.644148925756 and parameters: {'learning_rate': 0.0013855986361056194, 'weight_decay': 2.2800681297534146e-07, 'num_layers': 3, 'num_neurons': 249, 'dropout_rate': 0.40022334857731096}. Best is trial 40 with value: 52275.761113540946.
[I 2024-03-21 20:48:37,757] Trial 59 finished with value: 53311.29171198162 and parameters: {'learning_rate':

[I 2024-03-21 20:53:36,192] Trial 84 finished with value: 52381.18501905049 and parameters: {'learning_rate': 0.0007857570942354781, 'weight_decay': 1.716465470165978e-08, 'num_layers': 4, 'num_neurons': 152, 'dropout_rate': 0.026709970678652876}. Best is trial 40 with value: 52275.761113540946.
[I 2024-03-21 20:53:42,611] Trial 85 finished with value: 52345.51060024155 and parameters: {'learning_rate': 0.0010635803018984853, 'weight_decay': 9.59715124657824e-08, 'num_layers': 4, 'num_neurons': 194, 'dropout_rate': 0.041220106673301955}. Best is trial 40 with value: 52275.761113540946.
[I 2024-03-21 20:53:46,120] Trial 86 finished with value: 52300.603132277545 and parameters: {'learning_rate': 0.0006710738494263418, 'weight_decay': 2.4773951051012193e-07, 'num_layers': 4, 'num_neurons': 103, 'dropout_rate': 0.011217673429284345}. Best is trial 40 with value: 52275.761113540946.
[I 2024-03-21 20:53:52,894] Trial 87 finished with value: 52494.52725761039 and parameters: {'learning_rate'

[I 2024-03-21 20:56:38,044] Trial 112 finished with value: 52298.62805083896 and parameters: {'learning_rate': 0.0002478323259532404, 'weight_decay': 4.824018640171585e-07, 'num_layers': 4, 'num_neurons': 278, 'dropout_rate': 0.0007590616801793921}. Best is trial 40 with value: 52275.761113540946.
[I 2024-03-21 20:56:48,644] Trial 113 finished with value: 52349.96573064781 and parameters: {'learning_rate': 0.0006224367575828991, 'weight_decay': 4.789846349230199e-07, 'num_layers': 4, 'num_neurons': 320, 'dropout_rate': 0.02332044401203666}. Best is trial 40 with value: 52275.761113540946.
[I 2024-03-21 20:56:58,996] Trial 114 finished with value: 52316.271426775056 and parameters: {'learning_rate': 0.00021674896664151104, 'weight_decay': 1.7495559405025711e-06, 'num_layers': 4, 'num_neurons': 279, 'dropout_rate': 0.0399033904291577}. Best is trial 40 with value: 52275.761113540946.
[I 2024-03-21 20:57:04,607] Trial 115 finished with value: 52302.950127119984 and parameters: {'learning_

...

Epoch 384, Test Loss: 53694.4100
Epoch 385, Test Loss: 53641.5218
Epoch 386, Test Loss: 53701.3919
Epoch 387, Test Loss: 53693.7473
Epoch 388, Test Loss: 53676.1203
Epoch 389, Test Loss: 53640.3693


Epochs:  98%|██████████████████████████████████████████████████████████████████████▋ | 393/400 [00:14<00:00, 26.73it/s]

Epoch 390, Test Loss: 53663.9642
Epoch 391, Test Loss: 53660.5389
Epoch 392, Test Loss: 53646.7521
Epoch 393, Test Loss: 53675.5575
Epoch 394, Test Loss: 53659.6253
Epoch 395, Test Loss: 53689.3131


Epochs: 100%|████████████████████████████████████████████████████████████████████████| 400/400 [00:14<00:00, 26.95it/s]

Epoch 396, Test Loss: 53649.4936
Epoch 397, Test Loss: 53677.6512
Epoch 398, Test Loss: 53702.9340
Epoch 399, Test Loss: 53646.7020
Epoch 400, Test Loss: 53656.3859
Minimal loss 53631.649 was observed on test data on 359 epoch
